2.4 Model Selection 소개
=====

사이킷런의 model_selection 모듈은 학습 데이터와 테스트 데이터 세트를 분리하거나 교차 검증 분할 및 평가, 그리고 Estimator의 하이퍼 파라미터 튜닝을 위한 다양한 함수와 클래스를 제공

## 2.4.1 학습/테스트 데이터 셋 분리 - train_test_split()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_label,pred))

이미 학습한 학습 데이터 세트를 기반으로 예측했기 때문에 정확도는 100%가 나옴.<br>
예측을 수행하는 데이터 세트는 테스트 데이터 세트로 수행해야 함. 이를 분리하기 위해 train_test_split()를 사용하여 원본 데이터 세트에서 학습/테스트 데이터 세트를 분리

**train_test_split() 파라미터**
* test_size : 전체 데이터에서 테스트 데이터 세트 크기를 얼마로 할 지 결정. default 0.25, 즉 25%
* train_size : 전체 데이터에서 학습용 데이터 세트 크기를 얼마로 할 지 결정. test_size파라미터를 통상적으로 사용하기 때문에 일반적으로 사용 안 함.
* shuffle : 데이터를 분리 전 데이터를 섞을 지 결정. default True. 데이터를 분산시켜 좀 더 효율적인 학습 및 테스트 데이터를 만드는 데 사용
* random_state : 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값. 여기서는 예제 실행 시 동일 데이터를 보기위해 부여

train_test_split()의 반환은 튜플 형태. 순서대로 학습용 데이터 피처, 테스트용 데이터 피처, 학습용 데이터 레이블, 테스트용 데이터 레이블 반환

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier( )
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.3, random_state=121)

print(X_train.shape)
print(X_test.shape)

In [ ]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

붓꽃 데이터는 총 150개의 데이터로 위 30%의 테스트 데이터는 45개<br>
학습을 위한 데이터의 양을 일정 수준 이상으로 보장하는 것도 중요하지만, 학습된 모델에 대해 다양한 데이터를 기반으로 예측 성능을 평가하는 것도 매우 중요함

## 2.4.2 교차 검증

학습 데이터와 이에 대한 예측 성능을 평가하기 위한 별도의 테스트용 데이터가 필요하지만 이 방법 역시 과적합(Overfitting)에 취약한 약점을 가질 수 있음.

### 과적합(Overfitting)
모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 경우에 예측 성능이 과도하게 떨어지는 현상

고정된 학습 데이터와 테스트 데이터로 평가를 하다 보면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 생겨 다른 테스트용 데이터가 들어올 경우 성능이 저하 됨.
이를 위해 교차 검증을 사용하여 다양한 학습과 평가를 수행

대부분의 ML 모델의 성능 평가는 교차 검증 기반으로 1차 평가를 한 뒤에 최종적으로 테스트 데이터세트에 적용해 평가하는 프로세스

![validationSet](http://tykimos.github.io/warehouse/2017-3-25-Dataset_and_Fit_Talk_3.png)

### K 폴드 교차 검증

K 폴드 교차 검증은 가장 보편적으로 사용되는 교차 검증 기법. K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴트 세트에 학습과 검증 평가를 반복적으로 수행

![kfold](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile1.uf.tistory.com%2Fimage%2F99F84A3359CF93381DBAA7)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성.
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:',features.shape[0])


In [ ]:
n_iter = 0

# KFold객체의 split( ) 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환  
for train_index, test_index  in kfold.split(features):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    n_iter += 1
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 

### Stratified K 폴드

Stratified K 폴드는 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식.
불균형한 분포도를 가진 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것을 말함.

>예를 들어 대출 사기 데이터를 예측한다고 가정하면, 데이터가 1억 건이고, 수십 개의 피처와 대출 사기 여부 레이블(사기 :1, 정상 :0)으로 구성되어 있습니다. <br>이 중 대출 사기 레이블이 약 1000건이라고 하면 전체 중 0.0001%의 작은 확률로 대출 사기 레이블이 존재합니다. <br>이렇게 작은 비율로 사기 레이블 값이 있다면 K폴드로 랜덤하게 학습 및 검증 인덱스를 나누더라도 0과 1의 비율을 제대로 반영하지 못해 제대로 된 학습을 수행하지 못합니다.<br> 사기 레이블은 건수는 작지만 대출 사기를 예측할 수 있는 매우 중요한 피처 값들을 가지고 있기 때문에 원본 데이터와 유사한 대출 사기 레이블 값의 분포를 유지하는 것이 매우 중요합니다.

Stratified K 폴드는 이처럼 K 폴드가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제를 해결해 줌.

In [ ]:
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()

붓꽃 데이터의 레이블 값은 0, 1, 2 값 모두 50개로 동일함.
이슈 발생을 도출하기 위해 3개의 폴드 세트를 KFold로 생성하고, 교차 검증 시마다 생성되는 학습/검증 레이블 데이터 값의 분포를 확인해보자.

In [ ]:
kfold = KFold(n_splits=3)
# kfold.split(X)는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환. 
n_iter =0
for train_index, test_index  in kfold.split(iris_df):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    

위와 같은 유형으로 교차 검증 데이터 세트를 분할하면 검증 예측 정확도는 **0**이 될 수 밖에 없음.

StratifiedKFold를 사용하는 방법은 KFold를 사용하는 방법과 거의 비슷
큰 차이는 StratifiedKFold는 레이블 데이터 분포도에 따라 학습/검증 데이터를 나누기 때문에 split() 메서드에 인자로 **레이블 데이터 세트**도 추가적으로 입력해야함.

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

위 처럼 0, 1, 2 값이 각각 레이블 별로 동일하게 할당
이렇게 분할이 되어야 레이블 값 0, 1, 2를 각 폴드 별로 모두 학습할 수 있고, 이에 기반해 검증을 수행할 수 있음.

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)

    # 반복 시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.round(np.mean(scores), 4)) 

일반적으로 분류(Classification)에서의 교차 검증은 K폴드가 아니라 Stratified K 폴드로 분할돼야 합니다. 회귀(Regression)에서는 Stratified K 폴드가 지원되지 않습니다. 회귀의 결정값은 이산값 형태의 레이블이 아니라 연속된 숫자값이기 때문에 결정값별로 분포를 정하는 의미가 없기 때문

### 교차 검증을 보다 간편하게 - cross_val_score()
사이킷런은 교차 검증을 좀 더 편리하게 수행할 수 있게 해주는 cross_val_score() API를 제공

KFold로 데이터 학습/예측하는 코드의 프로세스
1. 폴드 세트를 설정
2. for 루프에서 반복으로 학습 및 테스트 데이터의 인덱스를 추출
3. 반복적으로 학습과 예측을 수행, 예측 성능을 반환

cross_val_score()는 이런 일련의 과정을 한꺼번에 수행
```Python
cross_val_score(estimator, X, y=None, scoring=None, cv=None, n_jobs=1, verbos=0, fit_params=None, pre_dispatch='2*n_jobs')
```

estimator는 사이킷런의 알고리즘 클래스를 의미하고, X는 피처 데이터 세트, y는 레이블 데이터 세트, scoring은 예측 성능 평가 지표를 기술하며, cv는 교차 검증 폴드 수를 의미. cross_val_score() 반환 값은 scoring 파라미터로 지정된 성능 지표 측정값을 배열 형태로 반환.
**classifier 클래스가 입력되면 Stratified K 폴드 방식을 사용하며, 회귀인 경우엔 K 폴드 방식으로 분할**

`교차 검증 폴드 수는 3, 성능 평가 지표는 정확도(accuracy)`

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 3개 
scores = cross_val_score(dt_clf , data , label , scoring='accuracy',cv=3)
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

### GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에

하이퍼 파라미터는 머신러닝 알고리즘을 구성하는 주요 구성 요소이며, 이 값을 조정해 알고리즘의 예측 성능을 개선할 수 있음.

사이킷런은 GridSearchCV API를 이용해 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안을 제공

> Grid는 격자라는 뜻으로, 촘촘하게 파라미터를 입력하면서 테스트를 하는 방식

DecisionTree 알고리즘의 여러 하이퍼 파라미터를 순차적으로 변경하며 최고 성능을 가지는 파라미터 조합을 찾고자 한다면 다음과 같이 파라미터의 집합을 만들고 순차적용하며 최적화를 수행할 수 있음.

```Python
grid_parameters = {
                    'max_depth' : [1,2,3],
                    'min_samples_split' : [2,3]
                  }
```

위와 같은 하이퍼 파라미터 집합을 적용하면 아래 표와 같이 순차적으로 적용되며, for 루프로 모든 파라미터를 번갈아 입력하면서 학습시키는 방법을 좀 더 유연하게 제공

|**순번**|**max_depth**|**min_samples_split**|
|:---:|:---:|:---:|
|1|1|2|
|2|1|3|
|3|2|2|
|4|2|3|
|5|3|2|
|6|3|3|

위와 같이 순차적으로 6회에 걸쳐 하이퍼 파라미터를 변경하면서 교차 검증 데이터 세트에 수행 성능을 측정.
CV(교차검증폴드수)가 3회라면 개별 파라미터 조합마다 3개의 폴딩 세트를 3회에 걸쳐 학습/평가해 평균값으로 성능을 측정하므로, 총 CV 3회 * 6개 파라미터 조합 = 18회의 학습/평가 수행

GridSearchCV 클래스의 생성자 파라미터
* estimator : classifier, regressor, pipeline
* param_grid : key + 리스트 값을 가지는 딕셔너리. estimator의 튜닝을 위해 파라미터명과 사용 될 파라미터 집합
* scoring : 예측 성능을 측정을 평가 방법. 사이킷런의 성능 평가 지표 문자열(ex. 정확도 - 'accuracy')
* cv : 교차 검증을 위해 분할되는 학습/테스트 세트의 개수
* refit : True일 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력 된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킴. default True.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩하고 학습데이타와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier(random_state=156)

### parameter 들을 dictionary 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [ ]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.  
### refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.  
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', \
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

* params 컬럼에는 수행할 때마다 적용된 개별 하이퍼 파라미터값
* rank_test_score는 하이퍼 파라미터 별로 성능이 좋은 score 순위를 나타냅니다. 1이 가장 뛰어난 순위이며, 이 때의 파라미터가 최적의 하이퍼 파라미터
* mean_test_score는 개별 하이퍼 파라미터별로 CV의 폴딩 테스트 세트에 대해 총 수행한 평가 평균값

GridSearchCV 객체의 fit()을 수행하면 최고 성능을 나타낸 하이퍼 파라미터의 값과 그 때의 평가 결과 값이 각각 best_params_, best_score_ 속성에 기록 됨.(즉, cv_results_ 의 rank_test_score가 1일 때의 값)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))